In [7]:
import requests
import pandas as pd
from pathlib import Path
from api import pet_api_key
from api import secret_api



In [2]:
# Pulling API - Testing data
def get_auth_token():
    """Authenticate with the Petfinder API and return the access token."""
    url = "https://api.petfinder.com/v2/oauth2/token"
    data = {
        "grant_type": "client_credentials",
        "client_id": pet_api_key,
        "client_secret": secret_api,
    }
    response = requests.post(url, data=data)
    response.raise_for_status()  # Ensure the request was successful
    return response.json()["access_token"]

def count_adoptable_dogs(city, token):
    """Count the total number of adoptable dogs in a specified city."""
    url = "https://api.petfinder.com/v2/animals"
    headers = {"Authorization": f"Bearer {token}"}
    params = {
        "type": "Dog",
        "location": city,
        "limit": 100,  # Adjust as needed, up to the API's max limit
    }
    total_dogs = 0
    page = 1  # Start from the first page

    while True:
        params["page"] = page  # Update the page parameter for each request
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Ensure the request was successful
        data = response.json()
        total_dogs += len(data["animals"])  # Add count of dogs in the current page

        # Check if we've reached the last page
        if page >= data["pagination"]["total_pages"]:
            break  # Exit loop if on the last page
        page += 1  # Move to the next page

    return total_dogs

# Authenticate and get the total number of adoptable dogs in New York City
token = get_auth_token()
total_dogs_nyc = count_adoptable_dogs("New York City, NY", token)
print(f"Total number of adoptable dogs in New York City: {total_dogs_nyc}")

Total number of adoptable dogs in New York City: 10628


In [5]:
# Pull in Petfinder API credentials for csv

def get_auth_token():
    """Authenticate with the Petfinder API and return the access token."""
    url = "https://api.petfinder.com/v2/oauth2/token"
    data = {
        "grant_type": "client_credentials",
        "client_id": pet_api_key,
        "client_secret": secret_api,
    }
    response = requests.post(url, data=data)
    response.raise_for_status()  # Ensure the request was successful
    return response.json()["access_token"]

def fetch_adoptable_pets(cities, token):
    """Fetch adoptable pets (cats and dogs) information for specified cities."""
    url = "https://api.petfinder.com/v2/animals"
    headers = {"Authorization": f"Bearer {token}"}
    pets_data = []

    for city in cities:
        for animal_type in ['Cat', 'Dog']:
            params = {
                "type": animal_type,
                "location": city,
                "limit": 100,
            }
            page = 1  # Start from the first page

            while True:
                params["page"] = page
                response = requests.get(url, headers=headers, params=params)
                response.raise_for_status()
                data = response.json()

                for animal in data["animals"]:
                    pets_data.append({
                        "type": animal["type"],
                        "breed": animal["breeds"]["primary"],
                        "location": animal["contact"]["address"]["city"],
                        "color": animal["colors"]["primary"] if animal["colors"]["primary"] else "N/A",
                        "size": animal["size"],
                        "date_posted": animal["published_at"],
                        "name": animal["name"],
                        "status": animal["status"],
                        "gender": animal["gender"]
                    })

                if page >= data["pagination"]["total_pages"]:
                    break
                page += 1

    return pd.DataFrame(pets_data)

# Cities to fetch data for
cities = ["New York City, NY", "Los Angeles, CA", "Chicago, IL", "Houston, TX", "Phoenix, AZ"]

# Authenticate
token = get_auth_token()

# Fetch pets data
pets_df = fetch_adoptable_pets(cities, token)

# Display the DataFrame
print(pets_df.head())

  type                 breed      location                   color    size  \
0  Cat   Domestic Short Hair      Sayville  Black & White / Tuxedo  Medium   
1  Cat  Domestic Medium Hair  East Hampton                   Black  Medium   
2  Cat   Domestic Short Hair  East Hampton                  Calico  Medium   
3  Cat  Domestic Medium Hair  East Hampton                   Black  Medium   
4  Cat  Domestic Medium Hair  East Hampton                   Black  Medium   

                date_posted       name     status  gender  
0  2024-02-15T01:55:03+0000      Bubba  adoptable    Male  
1  2024-02-15T01:49:20+0000   Rosemary  adoptable  Female  
2  2024-02-15T01:49:20+0000     Nutmeg  adoptable  Female  
3  2024-02-15T01:49:20+0000  Cranberry  adoptable  Female  
4  2024-02-15T01:49:20+0000   Cinnamon  adoptable  Female  


In [8]:
# Convert pet adoption data into Pandas Dataframe
adoption_data_df = pd.DataFrame(pets_df)
adoption_data_df.head()

,type,breed,location,color,size,date_posted,name,status,gender
0,Cat,Domestic Short Hair,Sayville,Black & White / Tuxedo,Medium,2024-02-15T01:55:03+0000,Bubba,adoptable,Male
1,Cat,Domestic Medium Hair,East Hampton,Black,Medium,2024-02-15T01:49:20+0000,Rosemary,adoptable,Female
2,Cat,Domestic Short Hair,East Hampton,Calico,Medium,2024-02-15T01:49:20+0000,Nutmeg,adoptable,Female
3,Cat,Domestic Medium Hair,East Hampton,Black,Medium,2024-02-15T01:49:20+0000,Cranberry,adoptable,Female
4,Cat,Domestic Medium Hair,East Hampton,Black,Medium,2024-02-15T01:49:20+0000,Cinnamon,adoptable,Female


In [18]:
#Export pet adoption data into a csv
adoption_data_df.to_csv("C:/Users/14407/Documents/STAK_Project_One/Resources/adoption.csv", index_label="Animal_ID")


In [19]:
#Read saved data
adoption_data_df = pd.read_csv("C:/Users/14407/Documents/STAK_Project_One/Resources/adoption.csv", index_col="Animal_ID")

#Display sample data
adoption_data_df.head()

,type,breed,location,color,size,date_posted,name,status,gender
Animal_ID,,,,,,,,,
0,Cat,Domestic Short Hair,Sayville,Black & White / Tuxedo,Medium,2024-02-15T01:55:03+0000,Bubba,adoptable,Male
1,Cat,Domestic Medium Hair,East Hampton,Black,Medium,2024-02-15T01:49:20+0000,Rosemary,adoptable,Female
2,Cat,Domestic Short Hair,East Hampton,Calico,Medium,2024-02-15T01:49:20+0000,Nutmeg,adoptable,Female
3,Cat,Domestic Medium Hair,East Hampton,Black,Medium,2024-02-15T01:49:20+0000,Cranberry,adoptable,Female
4,Cat,Domestic Medium Hair,East Hampton,Black,Medium,2024-02-15T01:49:20+0000,Cinnamon,adoptable,Female
